# 模型定义

我们先使用 Pytorch 定义一个经典的图像分类模型，包括卷积层、全连接层、激活函数，以及批归一化层。


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.conv1 = nn.Conv2d(1,16,kernel_size=3,padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16,32,kernel_size=3,padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.fc1 = nn.Linear(32*7*7,128)
        self.fc2 = nn.Linear(128,10)
    def forward(self,x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x,2)
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x,2)

        x = x.view(x.size(0),-1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 模型转换


Pytorch 模型转换为 ONNX 格式的过程中，可以使用多种方法来进行转换。

**使用 torch.onnx.export**

In [2]:
import torch.onnx

model = Model()

dummy_input = torch.randn(1,1,28,28)

# 基础转换
torch.onnx.export(model,dummy_input,"model.onnx")

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



- model：需要转换的 Pytorch 模型实例。
- dummy_input：用于模型的输入张量，用来追踪模型的计算图。
- "model.onnx"：导出的 ONNX 模型的保存路径。

In [7]:
# 此外，我们还可以增加参数来应对更加复杂的场景

torch.onnx.export(
    model,
    dummy_input,
    'model_2.onnx',
    export_params = True , # 默认True,导出模型参数
    opset_version = 11,  # ONNX opset 版本，默认为 11，支持最新的 ONNX 操作。
    do_constant_folding = True,  # 是否执行常量折叠优化。
    input_names=['input'],  # 输入的名称，用于标识输入节点。
    output_names=['output'],  # 输出的名称，用于标识输出节点。
    dynamic_axes={'input': {0: 'batch_size'}, 
                  'output': {0: 'batch_size'}},  # 动态轴设置，允许动态的 batch size。
)



================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



- export_params：控制是否导出模型参数。当设置为 False 时，只导出模型结构，而不包含权重。
- opset_version：指定导出使用的 ONNX opset 版本，确保与运行环境的兼容性。
- do_constant_folding：启用常量折叠优化，减少模型中的计算冗余。
- input_names 和 output_names：定义输入和输出节点的名称，方便在 ONNX 模型中查找。
- dynamic_axes：定义动态轴，这对于处理可变大小的输入（如 batch size，图像分割任务不限制图像尺寸等）非常有用。

# 转换检查

In [3]:
import onnx
onnx_model = onnx.load('model.onnx') # 加载模型

# 检查

onnx.checker.check_model(onnx_model)

使用 onnx.checker.check_model 检查 ONNX 模型时，如果模型正确，通常不会有任何输出。然而，如果模型存在错误或不一致性， check_model 会抛出一个 onnx.checker.ValidationError 异常，详细描述模型中的问题，如：

1. 图结构错误：节点的输入或输出不存在、存在循环依赖、节点的拓扑排序错误。
2. 数据类型不匹配：输入和输出之间的数据类型不一致、某个节点的输入数据类型与其操作不兼容。
3. 形状不匹配：节点的输入和输出的张量形状不匹配、某些操作（如矩阵乘法）要求特定形状，但输入张量不满足这些要求。
4. 不支持的操作：模型中包含不支持的或无效的操作符、操作符的属性配置错误。
5. 未定义的节点：模型中引用了未定义的节点或操作符。
6. 未定义的输入/输出：模型的输入或输出在模型中未正确定义。
7. 未能遵循 ONNX 标准：模型不符合 ONNX 规范，例如版本不一致、属性缺失等。

当 onnx.checker.check_model 发现这些问题时，它会抛出异常，并附带详细的错误信息，帮助开发者定位和修复问题。

# torch.onnx.export 的局限性

torch.onnx.export 方法是直接将模型从 Pytorch 转换为 ONNX 格式的主要方式，适用于大多数情况。然而，当模型包含一些动态行为或复杂的自定义操作时，方法 1 可能面临以下挑战：

1. 动态计算图：torch.onnx.export 默认的导出是基于一次前向传播过程（forward pass）捕获的计算图。这种静态追踪方法对大多数简单模型都有效，但对于包含动态分支或条件语句的模型，静态追踪可能会遗漏或错误处理某些路径，导致转换的模型不完整或不正确。
2. 自定义操作：某些 Pytorch 模型可能包含自定义操作，这些操作在 ONNX 的标准操作集中不存在。
torch.onnx.export的基础导出无法处理这些自定义操作，可能会导致转换失败或生成不符合预期的 ONNX 模型。

接下来，我们将进一步学习动态计算图的导出方法。

# 使用 torch.jit.script 与 torch.onnx.export

当模型包含一些动态操作时，如下定义的模型， forward 函数中包含一个条件判断，如果输入 x 的第一个元素大于 0，则执行 x = x * 2 操作。

In [13]:
class BranchModel(nn.Module):
    def __init__(self):
        super(BranchModel,self).__init__()
        self.fc1 = nn.Linear(10,20)
        self.fc2 = nn.Linear(20,1)
    def forward(self,x):
        x = torch.relu(self.fc1(x))
        if x[0,0]>0:
            x = x*2
        x = self.fc2(x)
        return x

In [14]:
branch_model = BranchModel()
dummy_input = torch.randn(1,10)
torch.onnx.export(branch_model,dummy_input,'branch_model.onnx')

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



C:\Users\change longer\AppData\Local\Temp\ipykernel_16280\1167482864.py:8: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x[0,0]>0:


In [15]:
model = onnx.load("branch_model.onnx")
for node in model.graph.node:
    print(f"Node name: {node.name}")  # 节点名称
    print(f"Node operation: {node.op_type}")  # 节点操作类型
    print(f"Node inputs: {node.input}")  # 节点输入
    print(f"Node outputs: {node.output}")  # 节点输出
    print("\n")

Node name: /fc1/Gemm
Node operation: Gemm
Node inputs: ['onnx::Gemm_0', 'fc1.weight', 'fc1.bias']
Node outputs: ['/fc1/Gemm_output_0']


Node name: /Relu
Node operation: Relu
Node inputs: ['/fc1/Gemm_output_0']
Node outputs: ['/Relu_output_0']


Node name: /Constant
Node operation: Constant
Node inputs: []
Node outputs: ['/Constant_output_0']


Node name: /Mul
Node operation: Mul
Node inputs: ['/Relu_output_0', '/Constant_output_0']
Node outputs: ['/Mul_output_0']


Node name: /fc2/Gemm
Node operation: Gemm
Node inputs: ['/Mul_output_0', 'fc2.weight', 'fc2.bias']
Node outputs: ['9']




In [16]:
# 通过 torch.jit.trace 进行追踪
traced_model = torch.jit.script(branch_model)

# 导出为 ONNX
torch.onnx.export(traced_model, dummy_input, "traced_model.onnx")

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [17]:
model = onnx.load("traced_model.onnx")
for node in model.graph.node:
    print(f"Node name: {node.name}")  # 节点名称
    print(f"Node operation: {node.op_type}")  # 节点操作类型
    print(f"Node inputs: {node.input}")  # 节点输入
    print(f"Node outputs: {node.output}")  # 节点输出
    print("\n")

Node name: /Constant
Node operation: Constant
Node inputs: []
Node outputs: ['/Constant_output_0']


Node name: /fc1/Gemm
Node operation: Gemm
Node inputs: ['x.1', 'fc1.weight', 'fc1.bias']
Node outputs: ['/fc1/Gemm_output_0']


Node name: /Relu
Node operation: Relu
Node inputs: ['/fc1/Gemm_output_0']
Node outputs: ['/Relu_output_0']


Node name: /Gather
Node operation: Gather
Node inputs: ['/Relu_output_0', '/Constant_output_0']
Node outputs: ['/Gather_output_0']


Node name: /Gather_1
Node operation: Gather
Node inputs: ['/Gather_output_0', '/Constant_output_0']
Node outputs: ['/Gather_1_output_0']


Node name: /Constant_1
Node operation: Constant
Node inputs: []
Node outputs: ['/Constant_1_output_0']


Node name: /Greater
Node operation: Greater
Node inputs: ['/Gather_1_output_0', '/Constant_1_output_0']
Node outputs: ['/Greater_output_0']


Node name: /Cast
Node operation: Cast
Node inputs: ['/Greater_output_0']
Node outputs: ['/Cast_output_0']


Node name: /If
Node operation: If
N

 torch.onnx.export 提供了一个简单有效的工具来进行模型转换，适用于大多数静态模型。然而，当模型中包含动态操作或自定义操作时，我们需要利用 torch.jit.script 来生成更精确的计算图，从而确保 ONNX 模型的完整性和正确性。最后，我们还需要使用 onnx.checker 进行模型有效性检查的重要性，以发现并解决潜在的问题。